In [1]:
import mlflow

In [2]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [7]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

### Track Experiments

In [8]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [9]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [10]:

import mlflow.sklearn
import mlflow.xgboost

In [12]:
for i, element in enumerate(models):
    model_name = element[0] #model name
    params = element[1]
    model = element[2] # actual model
    print(params)

{'C': 1, 'solver': 'liblinear'}
{'n_estimators': 30, 'max_depth': 3}
{'use_label_encoder': False, 'eval_metric': 'logloss'}
{'use_label_encoder': False, 'eval_metric': 'logloss'}


In [14]:
for i, element in enumerate(models):
    report = reports[i]
    print(report['accuracy'], element[0])

0.9166666666666666 Logistic Regression
0.9633333333333334 Random Forest
0.9766666666666667 XGBClassifier
0.9633333333333334 XGBClassifier With SMOTE


In [22]:
models[0]

('Logistic Regression',
 {'C': 1, 'solver': 'liblinear'},
 LogisticRegression(C=1, solver='liblinear'),
 (array([[ 1.18673836,  1.51144074,  0.78490373, ..., -0.61229492,
          -0.13830257, -0.24753395],
         [-1.28810271, -1.03855344, -2.07092052, ...,  0.49607021,
          -1.50376955,  0.62474155],
         [ 1.66393774,  1.55142135,  2.25024183, ..., -0.69955586,
           1.36600648, -0.68290518],
         ...,
         [ 0.43101615,  0.90013637, -0.42606094, ..., -0.32069604,
          -1.01508454,  0.11782042],
         [ 0.79839935,  1.5003473 , -0.45098948, ..., -0.54728572,
          -1.42140103,  0.11944858],
         [ 0.67367695,  1.27538516, -0.39960348, ..., -0.464427  ,
          -1.22522394,  0.10635794]], shape=(700, 10)),
  array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0,

In [23]:
len(models[0])

5

In [25]:
for model_name,p,_,_,_ in models:
    if model_name == "XGBClassifier":
        print(p)
    

{'use_label_encoder': False, 'eval_metric': 'logloss'}


In [31]:
for i, element in enumerate(models):
    model_name = element[0] #model name
    params = element[1] # hyper parameters
    model = element[2] # actual model
    report = reports[i]
    if "XGB" in model_name:
        print(model_name,'*'*20,'\n',params,'\n','Accuracy: ','\n',report['accuracy'])

XGBClassifier ******************** 
 {'use_label_encoder': False, 'eval_metric': 'logloss'} 
 Accuracy:  
 0.9766666666666667
XGBClassifier With SMOTE ******************** 
 {'use_label_encoder': False, 'eval_metric': 'logloss'} 
 Accuracy:  
 0.9633333333333334


In [ ]:

mlflow.set_experiment("Anomaly Detection_with_BalancedData")
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

for i, element in enumerate(models):
    model_name = element[0] #model name
    params = element[1] # hyper parameters
    model = element[2] # actual model
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_0':report['0']['recall'],
            'recall_class_1':report['1']['recall'],
            'f1_score_class_0':report['0']['f1-score'],
            'f1_score_class_1':report['1']['f1-score'],   
            'f1_score_macro':report['macro avg']['f1-score'], 
            'recall_score_macro':report['macro avg']['recall'], 
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

### Register the Model

In [ ]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model_name'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

### Load the Model

In [41]:
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production

In [42]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

<ModelVersion: aliases=[], creation_timestamp=1722288798033, current_stage='None', description='', last_updated_timestamp=1722288798033, name='anomaly-detection-prod', run_id='3e6b0bdb88ac4df9add7a826594d8d33', run_link='', source='models:/XGB-Smote/1', status='READY', status_message='', tags={}, user_id='', version='1'>

In [43]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 
reference: codebasics
